In [1]:
from pygltflib import GLTF2, Accessor, Skin, Node

In [2]:
# Download this file from https://paste.c-net.org/ScreechGrubby
gltf = GLTF2.load("C:/Users/Philipp/Desktop/gitProjects/visualkinematics/pythonAnsatz/assets/generator.glb")

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/Philipp/Desktop/gitProjects/visualkinematics/pythonAnsatz/assets/generator.glb'

In [1]:
from enum import IntEnum
import struct

class GLTFComponentType(IntEnum):
    BYTE = 5120
    UNSIGNED_BYTE = 5121
    SHORT = 5122
    UNSIGNED_SHORT = 5123
    UNSIGNED_INT = 5125
    FLOAT = 5126

GLTF_COMPONENTTYPE_SIZES = {
    GLTFComponentType.BYTE: 1,
    GLTFComponentType.UNSIGNED_BYTE: 1,
    GLTFComponentType.SHORT: 2,
    GLTFComponentType.UNSIGNED_SHORT: 2,
    GLTFComponentType.UNSIGNED_INT: 4,
    GLTFComponentType.FLOAT: 4
}

GLTF_ACCESSORTYPE_COUNTS = {
    "SCALAR": 1,
    "VEC2": 2,
    "VEC3": 3,
    "VEC4": 4,
    "MAT2": 4,
    "MAT3": 9,
    "MAT4": 16
}

GLTF_COMPONENT_UNPACK_FORMATS = {
    GLTFComponentType.BYTE: "b",
    GLTFComponentType.UNSIGNED_BYTE: "B",
    GLTFComponentType.SHORT: "h",
    GLTFComponentType.UNSIGNED_SHORT: "H",
    GLTFComponentType.UNSIGNED_INT: "I",
    GLTFComponentType.FLOAT: "f"
}

def get_dense_data(gltf: GLTF2, accessor: Accessor):
    bufferView = gltf.bufferViews[accessor.bufferView]
    buffer = gltf.buffers[bufferView.buffer]
    buffer_data = gltf.get_data_from_buffer_uri(buffer.uri)
    result = []
    elem_stride = GLTF_ACCESSORTYPE_COUNTS[accessor.type] * GLTF_COMPONENTTYPE_SIZES[int(accessor.componentType)]
    for i in range(accessor.count):
        index = bufferView.byteOffset + accessor.byteOffset + i * elem_stride
        base64_elem_data = buffer_data[index:index + elem_stride]
        elem_data = struct.unpack(f"<{GLTF_COMPONENT_UNPACK_FORMATS[accessor.componentType] * GLTF_ACCESSORTYPE_COUNTS[accessor.type]}", base64_elem_data)
        if len(elem_data) == 1:
            result.append(elem_data[0])
        else:
            result.append(elem_data)
            
    return result

NameError: name 'GLTF2' is not defined

In [8]:
primitive = gltf.meshes[gltf.scenes[gltf.scene].nodes[0]].primitives[0]

indices = get_dense_data(gltf, gltf.accessors[primitive.indices])
positions = get_dense_data(gltf, gltf.accessors[primitive.attributes.POSITION])
normals = get_dense_data(gltf, gltf.accessors[primitive.attributes.NORMAL])
texcoords = get_dense_data(gltf, gltf.accessors[primitive.attributes.TEXCOORD_0])
joints = get_dense_data(gltf, gltf.accessors[primitive.attributes.JOINTS_0])
joint_weights = get_dense_data(gltf, gltf.accessors[primitive.attributes.WEIGHTS_0])
inverse_bind_matrices = get_dense_data(gltf, gltf.accessors[gltf.skins[0].inverseBindMatrices])


TypeError: list indices must be integers or slices, not NoneType

In [ ]:
from pythreejs import *
from IPython.display import display

In [ ]:
def rgba_to_html_color(rgba_tuple):
    # Convert each RGBA value to its corresponding 8-bit integer representation
    r = int(rgba_tuple[0] * 255)
    g = int(rgba_tuple[1] * 255)
    b = int(rgba_tuple[2] * 255)
    a = int(rgba_tuple[3] * 255)

    # Format the values as a hexadecimal color string
    color_string = "#{:02X}{:02X}{:02X}{:02X}".format(r, g, b, a)

    return color_string

In [ ]:
def create_material(material: Material):
    result = MeshPhongMaterial(
        color = rgba_to_html_color(material.pbrMetallicRoughness.baseColorFactor),
        metallicFactor = material.pbrMetallicRoughness.metallicFactor,
        roughnessFactor = material.pbrMetallicRoughness.roughnessFactor,
        emissiveFactor = material.emissiveFactor,
        skinning = True)

    return result

In [ ]:
mesh_material = create_material(gltf.materials[0])

In [ ]:
mesh_geometry = BufferGeometry()
mesh_geometry.attributes["position"] = BufferAttribute(positions)
mesh_geometry.attributes["index"] = BufferAttribute(indices)
mesh_geometry.attributes["uv"] = BufferAttribute(texcoords)
mesh_geometry.attributes["normal"] = BufferAttribute(normals)
mesh_geometry.attributes["skinIndex"] = BufferAttribute(joints)
mesh_geometry.attributes["skinWeight"] = BufferAttribute(joint_weights)

NameError: name 'positions' is not defined

In [ ]:
from typing import List

def create_skeleton(gltf: GLTF2, skin: Skin, bones: List[Bone]):
    def create_bone(gltf: GLTF2, node: Node):
        bone = Bone(name = node.name, 
                    position = node.translation, 
                    scale = node.scale, 
                    quaternion = node.rotation)
        if bones is not None:
            bones.insert(0, bone)
        for child_node in node.children:
            child_bone = create_bone(gltf, gltf.nodes[child_node])
            bone.add(child_bone)

        return bone

    return create_bone(gltf, gltf.nodes[skin.skeleton])

In [ ]:
bones = []
root_bone = create_skeleton(gltf, gltf.skins[0], bones)
skeleton = Skeleton(bones = bones, boneInverses = inverse_bind_matrices)

IndexError: list index out of range

In [ ]:
mesh = SkinnedMesh(mesh_geometry, mesh_material)
mesh.add(root_bone)
mesh.skeleton = skeleton

helper = SkeletonHelper(mesh)

NameError: name 'root_bone' is not defined

In [ ]:
sphere = Mesh(
    SphereBufferGeometry(1, 32, 16),
    MeshStandardMaterial(color='red')
)

In [ ]:
view_width = 800
view_height = 600

key_light = DirectionalLight(position=[0, 10, 10], intensity=0.6)
ambient_light = AmbientLight()

camera = PerspectiveCamera(position=[10, 6, 10], aspect=view_width/view_height)

# This line doesn't work, draws a small white rectangle
scene_elems = [camera, key_light, ambient_light, mesh, helper]

# This line works, draws a red sphere
# scene_elems = [camera, key_light, ambient_light, sphere]

scene = Scene(children = scene_elems)
renderer = Renderer(camera=camera, scene=scene,
                     controls=[OrbitControls(controlling=camera)],
                     width=view_width, height=view_height)
renderer

NameError: name 'helper' is not defined